## 知識情報学第6回演習サンプルプログラム ex6.ipynb
- Programmed by Wu Hongle, 監修　福井健一
- Last updated: 2019/09/02
- Checked with Python 3.8.8, scikit-learn 1.0
- MIT Lisence

## SVMによるBreast Cancerデータの識別
- 入れ子交差検証で最適パラメータを探索

In [23]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import scale
import optuna
optuna.logging.disable_default_handler()

### Breast Cancerデータのロード

In [4]:
df = load_breast_cancer()
X = df.data
y = df.target

# z標準化
X = scale(X)

In [5]:
print(df.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

### 入れ子交差検証でハイパーパラメータを最適化
- 【課題1】探索するパラメータにカーネル関数の追加や範囲を変更して最適パラメータを探してみましょう
    - グリッドサーチパラメータリストの書き方は下記を参照
        - https://scikit-learn.org/stable/modules/grid_search.html#grid-search
    - SVCの可能なパラメータリストは下記を参照
        - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
- 【課題2】Optunaを利用してハイパーパラメータを最適化し，グリッドサーチと比較してみましょう．
    - Optuna: https://optuna.org
    - 使い方は，Code Exmaplesを参照
    - グリッドサーチ同様に入れ子の交差検証を用いること
    - optunaでパラメータの生成範囲指定は下記を参照（関数 suggest_***）
        - https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial
- 【課題3】最適なカーネル関数およびハイパーパラメータ，そこから分かるデータの特徴について考察してみましょう．

### 課題１
グリッドサーチするパラメータにC、kernelを追加

In [26]:
# 外側ループのための交差検証用データ生成インスタンス
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

acc_trn_list = []  # 外側ループのfold毎の学習データに対するaccuracy格納用
acc_tst_list = []  # 外側ループのfold毎のテストデータに対するaccuracy格納用

# グリッドサーチのパラメータリスト
parameters = {
    "gamma": [0.01, 0.02, 0.05, 0.1, 0, 2, 1, 10, 100],
    "C": [0.1, 1, 10, 100, 1000],
    "kernel": ["linear", "rbf", "poly", "sigmoid"]
}
# 内側ループでグリッドサーチを行う交差検証インスタンス
gs = GridSearchCV(SVC(), parameters, cv=2)

k = 0
for train_itr, test_itr in kfold.split(X, y):
    # 内側ループのグリッドサーチ
    gs.fit(X[train_itr], y[train_itr])
    print(
        "Fold #{:2d}; Best Parameter: {}, Accuracy: {:.3f}".format(
            k + 1, gs.best_params_, gs.best_score_
        )
    )
    acc_trn_list.append(gs.score(X[train_itr], y[train_itr]))
    acc_tst_list.append(gs.score(X[test_itr], y[test_itr]))
    k = k + 1

Fold # 1; Best Parameter: {'C': 10, 'gamma': 0.02, 'kernel': 'rbf'}, Accuracy: 0.979
Fold # 2; Best Parameter: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.971
Fold # 3; Best Parameter: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 4; Best Parameter: {'C': 1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 5; Best Parameter: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 6; Best Parameter: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.979
Fold # 7; Best Parameter: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}, Accuracy: 0.979
Fold # 8; Best Parameter: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}, Accuracy: 0.975
Fold # 9; Best Parameter: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}, Accuracy: 0.975
Fold #10; Best Parameter: {'C': 10, 'gamma': 0.02, 'kernel': 'rbf'}, Accuracy: 0.975


### 平均Accuracy

In [27]:
print('Training data: %1.3f' % np.mean(acc_trn_list))
print('Test data: %1.3f' % np.mean(acc_tst_list))

Training data: 0.987
Test data: 0.977


### 課題２
optunaを使用してパラメータ最適化

In [28]:
# 外側ループのための交差検証用データ生成インスタンス
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

acc_trn_list = []  # 外側ループのfold毎の学習データに対するaccuracy格納用
acc_tst_list = []  # 外側ループのfold毎のテストデータに対するaccuracy格納用

k = 0
for train_itr, test_itr in kfold.split(X, y):
    X_inr = X[train_itr]
    y_inr = y[train_itr]

    def objective(trial):
        # define the hyperparameters to optimize
        gamma = trial.suggest_float("gamma", 1e-2, 1e2, log=True)
        C = trial.suggest_float("C", 1e-1, 1e3, log=True)
        kernel = trial.suggest_categorical(
            "kernel", ["linear", "rbf", "poly", "sigmoid"]
        )

        # create the SVM model with the hyperparameters
        model = SVC(C=C, gamma=gamma, kernel=kernel)

        # perform cross-validation and return the mean accuracy
        kfold_inr = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
        acc_list = []
        for train_itr_inr, test_itr_inr in kfold_inr.split(X_inr, y_inr):
            model.fit(X_inr[train_itr_inr], y_inr[train_itr_inr])
            acc_list.append(model.score(X_inr[test_itr_inr], y_inr[test_itr_inr]))
        return np.mean(acc_list)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=180)
    print(
        "Fold #{:2d}; Best Parameter: {}, Accuracy: {:.3f}".format(
            k + 1, study.best_trial.params, study.best_value
        )
    )
    model = SVC(**study.best_trial.params)
    model.fit(X[train_itr], y[train_itr])
    acc_trn_list.append(model.score(X[train_itr], y[train_itr]))
    acc_tst_list.append(model.score(X[test_itr], y[test_itr]))
    k = k + 1

Fold # 1; Best Parameter: {'gamma': 0.040022185204178294, 'C': 0.33694021541730657, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 2; Best Parameter: {'gamma': 6.52928601005033, 'C': 0.23308089156493642, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 3; Best Parameter: {'gamma': 0.048381266242251884, 'C': 0.3232370135971585, 'kernel': 'sigmoid'}, Accuracy: 0.971
Fold # 4; Best Parameter: {'gamma': 18.798661416403412, 'C': 0.4213597259874516, 'kernel': 'linear'}, Accuracy: 0.973
Fold # 5; Best Parameter: {'gamma': 0.01062584478600713, 'C': 0.1837458915873546, 'kernel': 'linear'}, Accuracy: 0.975
Fold # 6; Best Parameter: {'gamma': 22.832029550440073, 'C': 0.2524741851247946, 'kernel': 'linear'}, Accuracy: 0.971
Fold # 7; Best Parameter: {'gamma': 64.39204156010764, 'C': 826.8667966851684, 'kernel': 'linear'}, Accuracy: 0.951
Fold # 8; Best Parameter: {'gamma': 0.951781678534629, 'C': 4.0411046114555065, 'kernel': 'linear'}, Accuracy: 0.963
Fold # 9; Best Parameter: {'gamma': 0.666797740899719

In [22]:
print('Training data: %1.3f' % np.mean(acc_trn_list))
print('Test data: %1.3f' % np.mean(acc_tst_list))

Training data: 0.984
Test data: 0.974


### 課題３
- Cは比較的小さいことからある程度誤分類を許すモデルになっており、マージン内にある程度データが位置していることが予想される
- 多くのfoldで最適なカーネルはlinearとなったことから、高次元空間へ写像せずとも分離しやすいシンプルな分布であると予想される